In [1]:
from datetime import datetime, timedelta
import pickle
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

from tldextract import extract
import langid

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50)

from newspaper import Article

<ipython-input-1-6e87932399ef>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


NOTE: Mediacloud news for state-specific collection includes news ABOUT the particular state, and as such, might sometimes contain news from national outlets where the article itself might be focused on the state itself. Most (if not all) of the collection is going to be sourced from local state-specific outlets, but not always. Currently, we are keeping all URLs within the obtained collection for the state, except those from nytimes and foxnews since those are our chosen national outlets. Trusting mediacloud's collections to be truly state-focused even when urls come from national outlets. 

In [2]:
start_date = datetime.strptime('2023-04-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-06-30', '%Y-%m-%d').date()

In [3]:
ohio_news = pd.read_csv('data/mediacloud_ohio.csv')
print(ohio_news.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2656 entries, 0 to 2655
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ap_syndicated         2656 non-null   bool   
 1   collect_date          2656 non-null   object 
 2   feeds                 0 non-null      float64
 3   guid                  2656 non-null   object 
 4   language              2656 non-null   object 
 5   media_id              2656 non-null   int64  
 6   media_name            2656 non-null   object 
 7   media_url             2656 non-null   object 
 8   metadata              2656 non-null   object 
 9   processed_stories_id  2656 non-null   int64  
 10  publish_date          2656 non-null   object 
 11  stories_id            2656 non-null   int64  
 12  story_tags            2656 non-null   object 
 13  title                 2656 non-null   object 
 14  url                   2656 non-null   object 
 15  word_count           

In [4]:
ohio_news[['language', 'media_name', 'media_url', 'publish_date', 'title', 'url']].tail(20)

,language,media_name,media_url,publish_date,title,url
2636,en,morningjournal.com,http://morningjournal.com/,2023-06-30 13:43:04,Maximize memory function with a nutrient-rich diet,https://www.morningjournal.com/2023/06/30/maximize-memory-function-with-a-nutrient-rich-diet/
2637,en,content.tribtoday.com,http://content.tribtoday.com/,2023-06-30 10:37:47,Fri. 10:30 a.m.: Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.tribtoday.com/news/latest-news/2023/06/fri-1030-a-m-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
2638,en,wdtn.com,http://www.wdtn.com,2023-06-30 16:00:17,Supreme Court lets stand ruling that protects people with gender dysphoria under disability law,https://www.wdtn.com/news/u-s-world/ap-us-news/ap-supreme-court-lets-stand-ruling-that-protects-people-with-gender-dysphoria-under-disability-law/
2639,en,tribtoday.com,http://www.tribtoday.com/#spider,2023-06-30 10:37:47,Fri. 10:30 a.m.: Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.tribtoday.com/news/latest-news/2023/06/fri-1030-a-m-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
2640,en,thecatholictelegraph.com,http://thecatholictelegraph.com/,2023-06-30 10:54:43,BREAKING: Supreme Court sides with web designer opposed to same-sex marriage,https://www.thecatholictelegraph.com/breaking-supreme-court-sides-with-web-designer-opposed-to-same-sex-marriage/92066
2641,en,lovelandmagazine.com,http://lovelandmagazine.com/,2023-06-30 16:31:55,Ohio Capital Journal wins seven Society of Professional Journalists awards,https://www.lovelandmagazine.com/ohio-capital-journal-wins-seven-society-of-professional-journalists-awards/
2642,en,Parma Sun Post,http://www.cleveland.com/parmasunpost/,2023-06-30 17:20:26,"Trans influencer Dylan Mulvaney says Bud Light never called her, despite ad backlash, bullying and threats",https://www.cleveland.com/business/2023/06/trans-influencer-dylan-mulvaney-says-bud-light-never-called-her-despite-ad-backlash-bullying-and-threats.html
2643,en,Plain Dealer,http://www.cleveland.com/plaindealer/,2023-06-30 17:20:26,"Trans influencer Dylan Mulvaney says Bud Light never called her, despite ad backlash, bullying and threats",https://www.cleveland.com/business/2023/06/trans-influencer-dylan-mulvaney-says-bud-light-never-called-her-despite-ad-backlash-bullying-and-threats.html
2644,en,West Shore Sun,http://www.cleveland.com/westshoresun/,2023-06-30 17:20:26,"Trans influencer Dylan Mulvaney says Bud Light never called her, despite ad backlash, bullying and threats",https://www.cleveland.com/business/2023/06/trans-influencer-dylan-mulvaney-says-bud-light-never-called-her-despite-ad-backlash-bullying-and-threats.html
2645,en,cleveland.com,http://www.cleveland.com/,2023-06-30 17:20:26,"Trans influencer Dylan Mulvaney says Bud Light never called her, despite ad backlash, bullying and threats",https://www.cleveland.com/business/2023/06/trans-influencer-dylan-mulvaney-says-bud-light-never-called-her-despite-ad-backlash-bullying-and-threats.html


In [5]:
relevant_news = ohio_news[~ohio_news['media_name'].isin(['Fox News', 'New York Times'])]
relevant_news = relevant_news[relevant_news['language']=='en']
relevant_news = relevant_news[['media_name', 'publish_date', 'title', 'url']]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2655 entries, 0 to 2655
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   media_name    2655 non-null   object
 1   publish_date  2655 non-null   object
 2   title         2655 non-null   object
 3   url           2655 non-null   object
dtypes: object(4)
memory usage: 103.7+ KB
None


In [6]:
relevant_news.publish_date = pd.to_datetime(relevant_news.publish_date)
relevant_news = relevant_news[relevant_news['publish_date'].dt.date >= start_date]
relevant_news = relevant_news[relevant_news['publish_date'].dt.date <= end_date]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2655 entries, 0 to 2655
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    2655 non-null   object        
 1   publish_date  2655 non-null   datetime64[ns]
 2   title         2655 non-null   object        
 3   url           2655 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 103.7+ KB
None


In [7]:
def get_article_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        subtitle = article.meta_description
        if subtitle == '' or subtitle == ' ':
            subtitle = None
        text = article.text
        if text == '' or text == ' ':
            text = None
    except:
        subtitle = None
        text = None
    return subtitle, text

In [8]:
subtitles, texts = [], []
all_urls = list(relevant_news['url'])
for url in tqdm(all_urls):
    subtitle, text = get_article_from_url(url)
    subtitles.append(subtitle)
    texts.append(text)
relevant_news['subtitle'] = subtitles
relevant_news['text'] = texts
print(relevant_news.info())

100%|██████████| 2655/2655 [28:38<00:00,  1.54it/s]  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2655 entries, 0 to 2655
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    2655 non-null   object        
 1   publish_date  2655 non-null   datetime64[ns]
 2   title         2655 non-null   object        
 3   url           2655 non-null   object        
 4   subtitle      1356 non-null   object        
 5   text          2131 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 145.2+ KB
None


In [9]:
relevant_news.to_csv('data/ohio_article_texts_and_info.csv',
                     index=False)